In [118]:
import torch.nn as nn
import torch

In [119]:
multihead_attn = nn.MultiheadAttention(embed_dim=200,  # E_q(E_q必须能整除num_heads)
                                       num_heads=5,  # 注意力的数目
                                       # 默认kdim=None(即kdim=embed_dim)
                                       kdim=100,  # E_k
                                       # 默认vdim=None(即vdim=embed_dim)
                                       vdim=50,  # E_v
                                       dropout=0.1  # Dropout probability on `attn_output_weights`
                                       )

mask = torch.arange(4)[None, :] < torch.tensor([2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4])[:, None]
mask = torch.unsqueeze(mask, 0)
mask = torch.repeat_interleave(mask, 40, dim=0)
mask = ~mask

query = torch.randn(14, 8, 200)  # query.shape=(L, N, E_q);L is the target sequence length
key = torch.randn(4, 8, 100)  # key.shape=(S, N, E_k);S is the source sequence length
value = torch.randn(4, 8, 50)  # value.shape=(S, N, E_v)
attn_output, attn_output_weights = multihead_attn(query=query,
                                                  key=key,
                                                  value=value,
                                                  # attn_mask.shape=(N * num_heads, L, S)
                                                  # a True value indicates that the corresponding position is not allowed to attend
                                                  attn_mask=mask)

In [120]:
# ★★★★★注意: pytorch输出的是头的平均注意力分数(tensorflow输出的是所有头的注意力分数)
"""
内部机制:
if need_weights:
    # average attention weights over heads
    attn_output_weights = attn_output_weights.view(bsz, num_heads, tgt_len, src_len)
    return attn_output, attn_output_weights.sum(dim=1) / num_heads
else:
    return attn_output, None
"""
# attn_output.shape=(L, N, E_q)
attn_output.shape

torch.Size([14, 8, 200])

In [121]:
# attn_output_weights.shape=(N, L, S)
attn_output_weights.shape

torch.Size([8, 14, 4])

In [122]:
print(mask[0, :, :])
print(attn_output_weights[0, :, :])

tensor([[False, False,  True,  True],
        [False, False,  True,  True],
        [False, False,  True,  True],
        [False, False,  True,  True],
        [False, False, False,  True],
        [False, False, False,  True],
        [False, False, False,  True],
        [False, False, False,  True],
        [False, False, False,  True],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False]])
tensor([[0.4631, 0.6135, 0.0000, 0.0000],
        [0.4181, 0.5873, 0.0000, 0.0000],
        [0.4142, 0.6970, 0.0000, 0.0000],
        [0.3883, 0.7228, 0.0000, 0.0000],
        [0.5364, 0.3238, 0.2351, 0.0000],
        [0.4114, 0.3255, 0.2799, 0.0000],
        [0.3952, 0.3840, 0.2464, 0.0000],
        [0.3306, 0.4838, 0.2259, 0.0000],
        [0.3868, 0.3762, 0.2670, 0.0000],
        [0.1457, 0.4392, 0.2390, 0.2213],
        [0.2471, 0.1886, 0.1975, 0.3660],
     

In [123]:
for i, j in multihead_attn.named_parameters():
    print(str(i) + ".shape=", j.shape)

q_proj_weight.shape= torch.Size([200, 200])
k_proj_weight.shape= torch.Size([200, 100])
v_proj_weight.shape= torch.Size([200, 50])
in_proj_bias.shape= torch.Size([600])
out_proj.weight.shape= torch.Size([200, 200])
out_proj.bias.shape= torch.Size([200])
